In [ ]:
import os
import sys
import requests
from datetime import datetime 
from multiprocessing import Pool
import numpy as  np
import pandas as pd
from pathlib import Path
import pickle
from tqdm import tqdm
from time import time
from typing import Dict, List, Optional
sys.path.append('../')
sys.path.append('../anmodel')
import matplotlib.pyplot as plt
import anmodel
from scipy.signal import argrelmax
from scipy.signal import argrelmin
from scipy.signal import periodogram
from scipy import signal

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

S=pd.read_csv("A.csv")

In [ ]:
#Current
#IUNaV. INaV. INaP, IUNaP, IKNa, IKvhh, ILeak, ICaV
VK=-100.0
VLeak=-60.95
VNa=55.0
VCa=120.0

#INaV
def am(v):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0) / (1.0-np.exp(-(v+33.0)/10.0))
def bm(v):
    return 4.0 * np.exp(-(v+53.7)/12.0)

def mna3(v):
    xxx1xxx=am(v)/(am(v)+bm(v))
    return xxx1xxx**3

#ILeak

def ILeak(g,v):
    return g*(v-VLeak)

def ILeakK(g,v):
    return g*0.6095*(v-VK)
def ILeakNa(g,v):
    return g*0.3905*(v)

#IKvhh

def IKvhh(g, v,n):
    return g*(n**4)*(v-VK)

#ICaV
def mca(v):
    return (1.0 / (1.0 + np.exp(-(v+20.0)/9.0)))**2

def ICaV(g, v):
    return g*mca(v)*(v-VCa)

#IKNa
def mna(na):
    return 1.0/(1.0+(32.0/na)**(3.0))


def IKNa(g,v,na):
    return g*mna(na)*(v-VK)

#IUNaV

def amuv(v,x):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0+x) / (1.0-np.exp(-(v+33.0+x)/10.0))
def bmuv(v,x):
    return 4.0 * np.exp(-(v+53.7+x)/12.0)

def mnauv3(v,x):
    xxx1xxx=amuv(v,x)/(amuv(v,x)+bmuv(v,x))
    return xxx1xxx**3

def IUNaV(g,v,h,x):
    return g*mnauv3(v,x)*h*(v-VNa)

In [ ]:
mnnan1=anmodel.models.NAN10model()

Size=0
XXX1=[]
XXX2=[]
XXX3=[]
XXX4=[]
XXX5=[]
XXX6=[]
XXX7=[]
XXX8=[]


for ii in tqdm(range(len(S["g_kvhh"]))):
    if ii==0:
        continue
    A=[]
    c=ii
    A.append(S['g_kvhh'][c])
    A.append(S['g_unav'][c])
    A.append(S['g_kna'][c])
    A.append(S['g_leak'][c])
    A.append(S['g_cav'][c])
    A.append(S['t_na'][c])
    A.append(S['x_na'][c])
    A.append(S['y_na'][c])
    

    BBB=['g_kvhh', 'g_unav', 'g_kna', 'g_leak','g_cav', 't_na','x_na','y_na']
    dic={key:  val for key, val in zip(BBB,A)}
    #new_params: pd.DataFrame=pd.DataFrame.from_dict(dic,orient='index').T
    new_params=dic
    mnnan1.set_params(new_params)
    s,info=mnnan1.run_odeint()
    
    f, spw = periodogram(s[19999:,0], fs=1000)
    maxamp: float = max(spw)
    nummax: int = spw.tolist().index(maxamp)
    maxfre: float = f[nummax]
    length=(1/maxfre)
    long=(30/length)*2+100
    
    X=s[20000:,3]
    maxid = signal.argrelmax(X, order=50) #Max
    minid=signal.argrelmin(X, order=50) #Min
    #A
    List=[]    #-90-60
    for i in range(1500):
        List.append([-110+i/10,0])

    for i in (range(1500)):
        dd=0
        for k in range(len(s[19999:,0])-1):
            p=-110+i/10
            if((s[19999:,0][k]-p)*(s[19999:,0][k+1]-p))<0:
                dd=dd+1

        List[int(np.floor((p+110)*10))][1]=dd

    Mark=-110
    for i in range(len(List)):
        if (List[i][1]>long):
            Mark=List[i][0]
            break
    Time=[]
    for i in range(len(s[19999:,0])-1):
             if((s[19999:,0][i]-Mark)*(s[19999:,0][i+1]-Mark))<0:
                Time.append(i)
    time=[]
    for i in range(len(Time)):
        if(len(s[19999:,0])-1-30<Time[i] or Time[i]<30):
            continue

        if(s[19999:,0][Time[i]-30]<Mark and  s[19999:,0][Time[i]-29]<Mark and  s[19999:,0][Time[i]-28]<Mark and  s[19999:,0][Time[i]-27]<Mark and  s[19999:,0][Time[i]-26]<Mark and  s[19999:,0][Time[i]-25]<Mark and  s[19999:,0][Time[i]-24]<Mark and  s[19999:,0][Time[i]-23]<Mark and  s[19999:,0][Time[i]-22]<Mark and  s[19999:,0][Time[i]-21]<Mark and s[19999:,0][Time[i]-20]<Mark and s[19999:,0][Time[i]-19]<Mark and  s[19999:,0][Time[i]-18]<Mark and  s[19999:,0][Time[i]-17]<Mark and  s[19999:,0][Time[i]-16]<Mark and s[19999:,0][Time[i]-15]<Mark and  s[19999:,0][Time[i]-14]<Mark and  s[19999:,0][Time[i]-13]<Mark and  s[19999:,0][Time[i]-12]<Mark and s[19999:,0][Time[i]-11]<Mark and s[19999:,0][Time[i]-10]<Mark and  s[19999:,0][Time[i]-9]<Mark and  s[19999:,0][Time[i]-8]<Mark and s[19999:,0][Time[i]-7]<Mark and  s[19999:,0][Time[i]-6]<Mark and s[19999:,0][Time[i]-5]<Mark and s[19999:,0][Time[i]-4]<Mark and  s[19999:,0][Time[i]-3]<Mark and s[19999:,0][Time[i]-2]<Mark  and s[19999:,0][Time[i]-1]<Mark and s[19999:,0][Time[i]-0]<=Mark and s[19999:,0][Time[i]+1]>Mark and  s[19999:,0][Time[i]+2]>Mark ):
            time.append([Time[i], 'u'])
        if( s[19999:,0][Time[i]-2]>Mark and s[19999:,0][Time[i]-1]>Mark and s[19999:,0][Time[i]-0]>=Mark and s[19999:,0][Time[i]+1]<Mark and  s[19999:,0][Time[i]+2]<Mark and  s[19999:,0][Time[i]+3]<Mark  and s[19999:,0][Time[i]+4]<Mark  and s[19999:,0][Time[i]+5]<Mark and s[19999:,0][Time[i]+6]<Mark  and s[19999:,0][Time[i]+7]<Mark   and s[19999:,0][Time[i]+8]<Mark   and s[19999:,0][Time[i]+9]<Mark   and s[19999:,0][Time[i]+10]<Mark  and s[19999:,0][Time[i]+11]<Mark  and s[19999:,0][Time[i]+12]<Mark  and s[19999:,0][Time[i]+13]<Mark  and s[19999:,0][Time[i]+14]<Mark  and s[19999:,0][Time[i]+15]<Mark  and s[19999:,0][Time[i]+16]<Mark  and s[19999:,0][Time[i]+17]<Mark  and s[19999:,0][Time[i]+18]<Mark  and s[19999:,0][Time[i]+19]<Mark  and s[19999:,0][Time[i]+20]<Mark  and s[19999:,0][Time[i]+21]<Mark  and s[19999:,0][Time[i]+22]<Mark  and s[19999:,0][Time[i]+23]<Mark  and s[19999:,0][Time[i]+24]<Mark  and s[19999:,0][Time[i]+25]<Mark  and s[19999:,0][Time[i]+26]<Mark and s[19999:,0][Time[i]+27]<Mark and s[19999:,0][Time[i]+28]<Mark and s[19999:,0][Time[i]+29]<Mark and s[19999:,0][Time[i]+30]<Mark ):
            time.append([Time[i],'d'])       

    aaa1=-1
    aaa2=-1
    label=0
    for i in range(len(time)):
        if(time[i][1]=='u'):
            label=label+1
            if(label==1):
                aaa1=time[i][0]
            if(label==6):
                aaa2=time[i][0]
        if(aaa2!=-1):
            break
    
    Vnorm=[]
    Nanorm=[]
    Kvhh=[]
    UNaV=[]
    KNa=[]
    Leak=[]
    LeakK=[]
    LeakNa=[]
    CaV=[]
    GG=[]
    
    for ll in range(aaa2-aaa1-1):
        xxxx1=aaa1+ll
        Kvhh.append(IKvhh(A[0],s[19999:,0][xxxx1],s[19999:,2][xxxx1]))
        UNaV.append(IUNaV(A[1],s[19999:,0][xxxx1],s[19999:,1][xxxx1], A[6]))
        KNa.append(IKNa(A[2],s[19999:,0][xxxx1], s[19999:,3][xxxx1]))
        Leak.append(ILeak(A[3],s[19999:,0][xxxx1]))
        LeakK.append(ILeakK(A[3],s[19999:,0][xxxx1]))
        LeakNa.append(ILeakNa(A[3],s[19999:,0][xxxx1]))
        CaV.append(ICaV(A[4],s[19999:,0][xxxx1]))
        
        Vnorm.append(s[19999:,0][xxxx1])
        Nanorm.append(s[19999:,3][xxxx1])
        
    
        GG.append(abs(Kvhh[ll])+abs(UNaV[ll])+abs(KNa[ll])+abs(LeakNa[ll])+abs(LeakK[ll])+abs(CaV[ll]))
        Kvhh[ll]=abs(Kvhh[ll])/GG[ll]
        UNaV[ll]=abs(UNaV[ll])/GG[ll]
        KNa[ll]=abs(KNa[ll])/GG[ll]
        LeakK[ll]=abs(LeakK[ll])/GG[ll]
        LeakNa[ll]=abs(LeakNa[ll])/GG[ll]
        CaV[ll]=abs(CaV[ll]/GG[ll])
        
        
        #gg=abs(KNa[ll])/GG
        gg=abs(LeakK[ll]/GG)

    
    
    
    D1=Kvhh
    D2=UNaV
    D3=KNa
    D4=LeakK
    D5=LeakNa
    D6=CaV
    
    D7=Vnorm
    D8=Nanorm
    
    
    t1=[]
    t2=[]
    t3=[]
    t4=[]
    t5=[]
    t6=[]
    
    t7=[]
    t8=[]
    
    MD1=[]
    MD2=[]
    MD3=[]
    MD4=[]
    MD5=[]
    MD6=[]
    MD7=[]
    MD8=[]
    '''
    if(len(D1)==0):
        Size=Size+1
        continue
        '''
    
    dt=10000/len(D1)
    for j in range(len(D1)):

        t7.append(dt*j)
        MD7.append(Vnorm[j])
        t8.append(dt*j)
        MD8.append(Nanorm[j])
    
    T=np.linspace(0,10000,10000)

    mmd7=np.interp(T,t7,MD7)
    XXX7.append(mmd7)
    mmd8=np.interp(T,t8,MD8)
    XXX8.append(mmd8)
       


In [ ]:
for i in tqdm(range(len(XXX8))):
    a1=min(XXX8[i])
    a2=max(XXX8[i])
    a3=a2-a1
    for j in range(len(XXX8[i])):
        XXX8[i][j]=XXX8[i][j]*10/a3
    b1=min(XXX8[i])
    for j in range(len(XXX8[i])):
        XXX8[i][j]=XXX8[i][j]-5-b1

In [ ]:
for i in tqdm(range(len(XXX7))):
    a1=min(XXX7[i])
    a2=max(XXX7[i])
    a3=a2-a1
    for j in range(len(XXX7[i])):
        XXX7[i][j]=XXX7[i][j]*10/a3
    b1=min(XXX7[i])
    for j in range(len(XXX7[i])):
        XXX7[i][j]=XXX7[i][j]-5-b1

In [ ]:
import seaborn as sns
plt.figure()
#plt.title("Normalized Voltage")
fff=sns.heatmap(XXX7,cmap='Spectral')
plt.axis("off")
plt.savefig("Fig1D1.png")

In [ ]:
plt.figure()
#plt.title("Normalized Na")
fff=sns.heatmap(XXX8,cmap='Spectral')
plt.axis("off")
plt.savefig("Fig1D2.png")